In [2]:
import scipy as sp
import numpy as np
import sympy as sy
import scipy.integrate as spi
from sympy.abc import symbols
import scipy.optimize as spo
from sympy.utilities.lambdify import lambdify 
import itertools as it
import time
import matplotlib.pyplot as plt
import iminuit as im

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
t,x,y, z, w, h, a,b,c,d, e, f, g,h = symbols('t,x,y, z, w, h, a,b,c,d, e,f,g,h', real = True)

In [5]:
#construct homotopy
def H(t, G, F, gamma):
    return [(1 - t)*G[i] + gamma*t*F[i] for i in range(len(G))]

#for degree 3 polynomial, do we need to generalise for more than one variable??
def G(x_array):
    func_listG = [i**3 - 1 for i in x_array]
    return func_listG

#construct inital starting systems for different number of starting systems
def F3(x_array):
    return [x_array[0]**3 - 6*x_array[1] + np.pi*x_array[2]**2, x_array[1]**3 + x_array[2]**2 + 8*x_array[0], x_array[2]**3 \
            + x_array[0]*x_array[1]*x_array[2] + np.pi]

def F4(x_array):
    return [x_array[0]**3 - 6*x_array[1] + np.pi*x_array[2]**2 + np.pi*x_array[3]**2, x_array[1]**3 + x_array[2]**2 + 8*x_array[0], x_array[2]**3 \
            + x_array[0]*x_array[1]*x_array[2] + np.pi, x_array[3]**3 - np.pi]

def F2predictortest(x_array):
    return [x_array[0] - 10, x_array[1] - 10]

def F2(x_array):
    return [x_array[0]**3 - 6*x_array[1] + np.pi, x_array[1]**3 + 8*x_array[0] + 76]

def F2Easy(x_array):
    return [x_array[0]**2  + np.pi, x_array[1]**2 - 76] 

def F1(x_array):
    return [x_array[0]**3 + np.pi + x_array[0]**2 - x_array[0]]

def gamma_generate():
    real = np.random.rand()
    im = np.sqrt(1- real**2)
    return real + im*1j

def G_roots_find(n):
    root_list = [1, np.exp(1j*2*np.pi/3), np.exp(1j*2*np.pi*2/3)]
    return [i for i in it.product(root_list, repeat = n)]

def split_function(func, expansion_array, expansion_variables):
    
    im_real_func = func(expansion_array)
    full_array = sum([abs(sy.re(i_re)) for i_re in im_real_func] + [abs(sy.im(i_im)) for i_im in im_real_func])
    
    full_func = lambdify(expansion_variables, full_array)
    
    return full_func


In [108]:
def Homotopy_Continuation(t, x_array, F, N, expansion_array, expansion_variables, tolerance = 1e-10, tolerance_zero = 1e-10, ratio_tolerance = 1e-5,\
                          N_ratio_tolerance = 50, debug = False, corrector_Newtons = True):
    
    time_start = time.time()
    
    #count the number of roots
    num = 0
    delta_t = 1/N
    n = len(x_array)
    gamma = gamma_generate()
    gamma= 0.6 + 0.8j
    G_roots = G_roots_find(n)
    
    #construct homotopy
    H_func = H(t, G(x_array), F(x_array), gamma)
    
    #first derivative of H
    H_diff_x = sy.Matrix([[H_func[i].diff(x_array[j]) for i in range(len(x_array))] for j in range(len(x_array))])
    

    determinant_H = H_diff_x.det()
    
    #derivative with respect to t
    H_diff_t = sy.Matrix([H_func[i].diff(t) for i in range(len(x_array))])
    
    #check determinant is not zero so can invert
    if abs(determinant_H) == 0:
        raise TypeError('The determinant of H is zero!')
    
    #function of determinant H
    det_H = lambdify((t, x_array), determinant_H)
    
    H_diff_x_inv = H_diff_x**-1
    H_diff_inv = -H_diff_x_inv*H_diff_t
    
    H_Hdiffprime = H_diff_x_inv*sy.Matrix(H_func)
    
    H_Hdiffprime = lambdify((t, x_array), [H_Hdiffprime[i] for i in range(len(H_Hdiffprime))])
        
    H_prime = lambdify((t, x_array), [H_diff_inv[i] for i in range(len(H_diff_inv))])
    
    Hprime_x = lambdify((x_array,t), [H_diff_x[i] for i in range(len(H_diff_x))])    
    H_func_1d = lambdify([x_array,t], H_func)
    

    x_old_arrays = []
    x_olds = []
    
    #run for all roots of starting system
    for x_old in G_roots:
        x_old_array = []
        num += 1
            
        t_n = 0
        
        #run for all steps starting at t=0 ending at t=1
        while round(t_n,5) < 1:
            x_old_array.append(x_old)
            t_old = t_n
            t_n += delta_t
            
            if n == 1:
                sol = spi.solve_ivp(H_prime, (t_old, t_n), x_old)
                sol_x = sol.y[-1][-1]
                
                H_func_1 = lambdify((x,t), H_func_1d([x], t)[0])
                Hprime_x_1 = lambdify((x,t), Hprime_x([x], t)[0])

                #x_old = [spo.newton(H_func_1, sol_x, fprime = Hprime_x_1, args=(t_n, ))]
            
            else:
                if abs(det_H(t_n, x_old)) < tolerance_zero:
                    raise TypeError('The determinant of H is zero!')
                
                #perform RK4 method
                x_old_predictor=x_old
                sol = spi.solve_ivp(H_prime, (t_old, t_n), x_old)
                sol_x = sol.y[:,-1]
                   
            #newton's method
            x_old = sol_x
            ratio = np.full(n, 1)
            N_ratio = 0
            delta = np.full(n, ratio_tolerance)
            
            if corrector_Newtons is True:
                
                time_newtons_start = time.time()
                #tolerance criteria for step size in Newton's Method
                while max(ratio) > ratio_tolerance and N_ratio < N_ratio_tolerance:
                    if debug: print("Before Newton", x_old)
                    if abs(det_H(t_n, x_old)) < tolerance_zero:
                        raise TypeError('The determinant of H is zero!')
                    x_old_intermediate = x_old - H_Hdiffprime(t_n, x_old)
                    delta_old = delta
                    delta = abs(x_old_intermediate - x_old)
                    ratio = [delta[j]/(delta_old[j] + 1e-10) for j in range(n)]
                    x_old = x_old_intermediate
                    N_ratio += 1
                    #x_old = spo.newton(H_func, sol_x, fprime = Hprime_x, args=(t_n, ))
                    
                    time_newtons_end = time.time()
                    if debug: print("After Newton", x_old)
                if debug:

                    print('Time for Newton: {}'.format(time_newtons_end - time_newtons_start))
                    
            else:
                time_minuit_start = time.time()
                #iminuit
                H_func_t = H(t_n, G(expansion_array), F(expansion_array), gamma)
               
                if debug: print("Homotopy at current step: ", H_func_t)
                
                #split real and imaginary
                H_im_real_array = sum([abs(sy.re(i_re)) for i_re in H_func_t] + [abs(sy.im(i_im)) for i_im in H_func_t])
                
                if debug: print("Homotopy Absolute value at current step: ", H_im_real_array)
                
                H_func_t_combined = lambdify([expansion_variables], H_im_real_array)

                x_old_re_im = []

                #split x_old to re and im
                for i in range(n):
                    x_old_re_im.append(np.real(x_old[i]))
                    x_old_re_im.append(np.imag(x_old[i]))
                    
                string_variables = [str(j) for j in expansion_variables]
                #call iminuit function
                if debug: print("Before Minuit we start at", x_old_re_im)
                    
                printlevel = 10 if debug else 0
                
                m = im.Minuit.from_array_func(H_func_t_combined, x_old_re_im, forced_parameters= string_variables,print_level=printlevel)
                m.migrad(resume=False)

                x_old_im_re_vals = m.values
                
                
                x_old = [x_old_im_re_vals[j] + 1j*x_old_im_re_vals[j+1] for j in range(0, 2*n, 2)]
                
                if debug: print("After Minuit we got", x_old)
                time_minuit_end = time.time()
                
                if debug:
                    print('Time for Minuit: {}'.format(time_minuit_end - time_minuit_start))

        #check root is found
        remainder = list(map(abs, F(x_old))) 

        if max(remainder) < tolerance:
            x_old = [x_old[i].real if abs(x_old[i].imag) < tolerance_zero else x_old[i] for i in range(len(x_old))]

            print('Root {}, \n{}, \nAccuracy {}!!\n'.format(num, x_old, remainder))
            x_olds.append(x_old)
            x_old_arrays.append(x_old_array)

    time_end = time.time()
    
    print('It took {} s to run!'.format(time_end - time_start))
    
    return x_olds, x_old_arrays

In [74]:
x_sols, x_sols_arrays = Homotopy_Continuation(t, [x,y,z], F3, N=2, expansion_array= [a + 1j*b, c + 1j*d, e + 1j*f], expansion_variables= [a,b, c,d,e,f], \
                                              tolerance = 10, N_ratio_tolerance= 10, ratio_tolerance= 1e-5,debug=False, corrector_Newtons = True)

Root 3, 
[(2.4015419329850514+0.9049586531040493j), (1.0084642611630346+2.515325925322324j), (-0.0031531161124764458+0.47972367788070025j)], 
Accuracy [1.1911096805381705, 1.3256677843398512, 0.2022301186303921]!!

Root 4, 
[(-2.3968256960864007+0.9468817202100097j), (-0.992412018419307+2.5036880899380276j), (0.051686246077659856-0.42100616771180754j)], 
Accuracy [1.9413975509336834, 1.8314426728320803, 0.3488339188640485]!!

Root 6, 
[(-0.9229107830847486+2.432370935914727j), (2.507749796172227-1.2909201566258932j), (-0.09633982881193362+0.49829151588185433j)], 
Accuracy [0.7591338858163416, 5.228016596997663, 0.6410419314234138]!!

Root 7, 
[(2.374107250607467-0.9457711284813672j), (1.049951423598628-2.5178532104178806j), (0.007329860673171476-0.45094344519321095j)], 
Accuracy [0.0937933228797495, 0.06601817436126639, 0.01111675558606413]!!

Root 8, 
[(0.964947994458278-2.446474233109535j), (-2.4252974069004924+0.9985915363547114j), (-0.010129185818374731+0.5521231600835743j)], 
Accu

KeyboardInterrupt: 

In [112]:
x_sols, x_sols_arrays = Homotopy_Continuation(t, [x,y], F2, N=2, expansion_array= [a + 1j*b, c + 1j*d], expansion_variables= [a,b, c,d], \
                                              tolerance = 1000, N_ratio_tolerance= 10, ratio_tolerance= 1e-5,debug=True, corrector_Newtons = False)

Homotopy at current step:  [0.5*(a + 1.0*I*b)**3 + (0.3 + 0.4*I)*(-6*c - 6.0*I*d + (a + 1.0*I*b)**3 + 3.14159265358979) - 0.5, 0.5*(c + 1.0*I*d)**3 + (0.3 + 0.4*I)*(8*a + 8.0*I*b + (c + 1.0*I*d)**3 + 76) - 0.5]
Homotopy Absolute value at current step:  Abs(2.4*a - 3.2*b + 0.8*c**3 - 1.2*c**2*d - 2.4*c*d**2 + 0.4*d**3 + 22.3) + Abs(3.2*a + 2.4*b + 0.4*c**3 + 2.4*c**2*d - 1.2*c*d**2 - 0.8*d**3 + 30.4) + Abs(-0.4*a**3 - 2.4*a**2*b + 1.2*a*b**2 + 0.8*b**3 + 2.4*c + 1.8*d - 1.25663706143592) + Abs(0.8*a**3 - 1.2*a**2*b - 2.4*a*b**2 + 0.4*b**3 - 1.8*c + 2.4*d + 0.442477796076938)
Before Minuit we start at [-1.7087847681474022, 0.0, -3.15094051384107, 0.0]
------------------------------------------------------------------
| FCN = 1.237E-05               |     Ncalls=988 (989 total)     |
| EDM = 2.14E-08 (Goal: 1E-05)  |            up = 1.0            |
------------------------------------------------------------------
|  Valid Min.   | Valid Param.  | Above EDM | Reached call limit |
-------

------------------------------------------------------------------
| FCN = 0.001961                |     Ncalls=889 (890 total)     |
| EDM = 3.33E-06 (Goal: 1E-05)  |            up = 1.0            |
------------------------------------------------------------------
|  Valid Min.   | Valid Param.  | Above EDM | Reached call limit |
------------------------------------------------------------------
|     False     |     True      |   True    |       False        |
------------------------------------------------------------------
| Hesse failed  |   Has cov.    | Accurate  | Pos. def. | Forced |
------------------------------------------------------------------
|     False     |     True      |   False   |   True    | False  |
------------------------------------------------------------------
After Minuit we got [(2.7475309735418922+1.143818729044069j), (2.1832166363028866+4.0679868854764045j)]
Time for Minuit: 0.03194904327392578
Root 5, 
[(2.7475309735418922+1.143818729044069j), (2.1

Homotopy at current step:  [(0.6 + 0.8*I)*(-6*c - 6.0*I*d + (a + 1.0*I*b)**3 + 3.14159265358979), (0.6 + 0.8*I)*(8*a + 8.0*I*b + (c + 1.0*I*d)**3 + 76)]
Homotopy Absolute value at current step:  Abs(4.8*a - 6.4*b + 0.6*c**3 - 2.4*c**2*d - 1.8*c*d**2 + 0.8*d**3 + 45.6) + Abs(6.4*a + 4.8*b + 0.8*c**3 + 1.8*c**2*d - 2.4*c*d**2 - 0.6*d**3 + 60.8) + Abs(-0.8*a**3 - 1.8*a**2*b + 2.4*a*b**2 + 0.6*b**3 + 4.8*c + 3.6*d - 2.51327412287183) + Abs(0.6*a**3 - 2.4*a**2*b - 1.8*a*b**2 + 0.8*b**3 - 3.6*c + 4.8*d + 1.88495559215388)
Before Minuit we start at [-2.2376871185013605, 1.26199698933151, 1.8524151269097948, 3.4802703989795822]
------------------------------------------------------------------
| FCN = 8.903E-05               |    Ncalls=1044 (1045 total)    |
| EDM = 3.63E-07 (Goal: 1E-05)  |            up = 1.0            |
------------------------------------------------------------------
|  Valid Min.   | Valid Param.  | Above EDM | Reached call limit |
-------------------------------------

In [110]:
x_sols, x_sols_arrays = Homotopy_Continuation(t, [x,y], F2, N=2, expansion_array= [a + 1j*b, c + 1j*d], expansion_variables= [a,b, c,d], \
                                              tolerance = 10, N_ratio_tolerance= 10, ratio_tolerance= 1e-5,debug=False, corrector_Newtons = True)

Root 1, 
[(-2.947125066959122+9.429222000461573e-06j), (-3.742642053614938+3.4587224114369064e-05j)], 
Accuracy [6.626484790520658e-05, 0.0021920131199718628]!!

Root 2, 
[(-2.270888819989243+1.6573772674573135j), (1.6911835292293993+3.5147051024473552j)], 
Accuracy [0.0026288674580881343, 0.004715660582324521]!!

Root 3, 
[(1.4892458865117097-2.7301836894639804j), (-4.47634465415171+0.36414763497141195j)], 
Accuracy [0.0005436795914851984, 0.0008123850508692704]!!

Root 4, 
[(-0.4923558472662283+2.8283529954460254j), (2.4732971869789178-3.4279076635805783j)], 
Accuracy [0.0020752424517227292, 0.003138324739659144]!!

Root 5, 
[(2.747529157356835+1.1438140867706104j), (2.1832177866058173+4.068006077870291j)], 
Accuracy [0.0010926572611605332, 0.0016699380899631913]!!

Root 6, 
[(1.4892619982008362+2.7302128884570327j), (-4.476325761864668-0.3641371452832883j)], 
Accuracy [0.0006587896177880659, 0.0008859091061783474]!!

Root 7, 
[(-2.270738627362966-1.6574504013483724j), (1.69105551772

In [ ]:
for i in range(0, 6, 2):
    print(i)

In [33]:
H_func_1d = lambdify((x_array,t), H_func)

NameError: name 'x_array' is not defined

In [22]:
def split_function(func, expansion_array, expansion_variables):
    
    im_real_func = func(expansion_array)
    full_array = sum([abs(sy.re(i_re)) for i_re in im_real_func] + [abs(sy.im(i_im)) for i_im in im_real_func])
    
    full_func = lambdify([expansion_variables], full_array)
    
    return full_func

In [23]:
F2real = split_function(F2Easy, [a + 1j*b, c + 1j*d], [a,b,c,d])

In [24]:
F2real([a,b,c,d])

2.0*Abs(a*b) + 2.0*Abs(c*d) + Abs(a**2 - 1.0*b**2 + 3.14159265358979) + Abs(-c**2 + 1.0*d**2 + 76)

In [25]:
hhh = abs(x**2 - 8 + y) + abs(y**2 - 16)

In [29]:
hhh = lambdify((x, y), hhh)

In [30]:
ggg = ['a','b','c','d']
m = im.Minuit(F2real, [1,2,3,4], forced_parameters= ggg, use_array_call=True)
m.migrad()
new_vals = m.values

In [31]:
new_vals[1]

-1.772453781598903

In [32]:
def tryh(x, y, z, g):
    return x+y+z+g

hi = x**2 + y**3 + b
array_y = [x,y]

In [ ]:
argg = [2, 3,4]

In [ ]:
tryh(1, *argg)

In [ ]:
p = 5.66  + 87j

In [ ]:
np.imag(p)

In [ ]:
tryggg = lambdify([x, y, b], hi)

In [ ]:
tryggg(1,2, 1)